In [29]:
import os
import io
import concurrent.futures
from datetime import datetime,timedelta
from minio import Minio

host = "127.0.0.1"
port = "9000"
usn = "admin"
pwd = "appTPDanube*2021*558"
bucket = "danube"
time_params = {
   "days":90
}

client = Minio(f"{host}:{port}", usn, pwd,secure=False)

def get_object(path):
    count = 0
    for cur_dir, dirs, files in os.walk(path):
        if 'xl.meta' in files:
            count += 1
            print(f'count: {count}')
            yield cur_dir,path

def get_object_key(cur_dir, path):
    return cur_dir.replace(path, '').strip('/')

def thread_run(func,workers=5, array:list=None):
    result = {}
    with concurrent.futures.ThreadPoolExecutor(max_workers=workers) as executor:
        print("begin: ")
        future_to_key = {executor.submit(func, *key):key for key in array}
        
        for future in concurrent.futures.as_completed(future_to_key):
            key = future_to_key[future]
            try:
                data = future.result()
            except Exception as exc:
                print('%r generated an exception: %s' % (key, exc))
            else:
                result[key] = data
                print(f"--{key} done {data}--")
    return result

def get_object_last_modify_time(bucket, key):
    result = client.get_object(bucket, key, length=1)
    # print(result.headers['Last-Modified'])
    return datetime.strptime(result.headers['Last-Modified'],'%a, %d %b %Y %H:%M:%S %Z')

def main(path):
    result = client.list_buckets()
    print(result[0].name)
    thread_run(compare_and_delete, 20, get_object(path))

def compare_and_delete(_d, _p):
    cur = datetime.now()
    key = get_object_key(_d, _p)
    print(cur - get_object_last_modify_time(bucket, key), key)
    if cur - get_object_last_modify_time(bucket, key) > timedelta(**time_params):
        print(f'delete {key}')
        client.remove_object(bucket, key)
        return key

if __name__ == "__main__":
    main('/app')



ksc
begin: 
('/app/hello0', '/app') generated an exception: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /ksc/hello1, request_id: 172ADABABE64AD87, host_id: 1c828266-e856-4c39-b437-17d375896a7c, bucket_name: ksc, object_name: hello1
('/app/hello2', '/app') generated an exception: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /ksc/hello1, request_id: 172ADABABE632E57, host_id: 1c828266-e856-4c39-b437-17d375896a7c, bucket_name: ksc, object_name: hello1
('/app/hello3', '/app') generated an exception: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /ksc/hello1, request_id: 172ADABABE6655AF, host_id: 1c828266-e856-4c39-b437-17d375896a7c, bucket_name: ksc, object_name: hello1
('/app/hello1', '/app') generated an exception: S3 operation failed; code: NoSuchKey, message: The specified key does not exist., resource: /ksc/hello1, request_id: 172ADABABE646E73, h

In [20]:
compare_and_delete('./hello1', '.')

In [32]:
def get_object_key(cur_dir, path):
    print(cur_dir.strip(path))
    return cur_dir.strip(path).strip('/')
get_object_key(*('/home/shurui/docker_data1/minio/data/1/test-bucket/hello863653614437489', '/home/shurui/docker_data1/minio/data/1/test-bucket'))

llo863653614437489


'llo863653614437489'